In [41]:
import yfinance as yf
import pandas as pd
import plotly.express as px
import numpy as np

In [52]:
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [53]:
msft = yf.Ticker("MSFT")
hist = msft.history(period="5y")

In [54]:

start_pos = 0
num_bars = 1000

fsma_period = 10
ssma_period = 100

In [55]:
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-11-14 00:00:00-05:00,78.460744,79.024533,77.972130,78.977554,18801300,0.00,0
2017-11-15 00:00:00-05:00,78.826462,79.034225,78.089856,78.363724,19383100,0.42,0
2017-11-16 00:00:00-05:00,78.477060,78.779257,78.325964,78.571495,20962800,0.00,0
2017-11-17 00:00:00-05:00,78.495918,78.495918,77.664869,77.815971,22079000,0.00,0
2017-11-20 00:00:00-05:00,77.815986,77.995411,77.674329,77.938751,16315000,0.00,0


In [56]:
# Requesting historical data

df = pd.DataFrame(hist)[[ 'Open', 'High', 'Low', 'Close']]
#df['Date'] = pd.to_datetime(df['Date'], unit='s')

df['fast_sma'] = df['Close'].rolling(fsma_period).mean()
df['slow_sma'] = df['Close'].rolling(ssma_period).mean()

# finding crossovers
df['prev_fast_sma'] = df['fast_sma'].shift(1)

df['time'] = df.index

df.dropna(inplace=True)
df

,Open,High,Low,Close,fast_sma,slow_sma,prev_fast_sma,time
Date,,,,,,,,
2018-04-10 00:00:00-04:00,87.660080,88.504516,86.948476,88.124992,86.051862,84.313706,86.137256,2018-04-10 00:00:00-04:00
2018-04-11 00:00:00-04:00,87.299541,88.514010,86.796676,87.157219,86.278626,84.395502,86.051862,2018-04-11 00:00:00-04:00
2018-04-12 00:00:00-04:00,87.698050,89.339486,87.698050,88.789177,86.676176,84.499757,86.278626,2018-04-12 00:00:00-04:00
2018-04-13 00:00:00-04:00,89.235092,89.358434,87.707516,88.314751,86.847909,84.597189,86.676176,2018-04-13 00:00:00-04:00
2018-04-16 00:00:00-04:00,89.254074,89.813873,88.637350,89.348953,87.383984,84.712519,86.847909,2018-04-16 00:00:00-04:00
...,...,...,...,...,...,...,...,...
2022-11-08 00:00:00-05:00,228.699997,231.649994,225.839996,228.869995,226.672000,255.139076,228.851001,2022-11-08 00:00:00-05:00
2022-11-09 00:00:00-05:00,227.369995,228.630005,224.330002,224.509995,225.990999,254.912922,226.672000,2022-11-09 00:00:00-05:00
2022-11-10 00:00:00-05:00,235.429993,243.330002,235.000000,242.979996,227.613998,254.810696,225.990999,2022-11-10 00:00:00-05:00


In [57]:
def find_crossover(fast_sma, prev_fast_sma, slow_sma):
    
    if fast_sma > slow_sma and prev_fast_sma < slow_sma:
        return 'bullish crossover'
    elif fast_sma < slow_sma and prev_fast_sma > slow_sma:
        return 'bearish crossover'
    
    return None


df['crossover'] = np.vectorize(find_crossover)(df['fast_sma'], df['prev_fast_sma'], df['slow_sma'])

signal = df[df['crossover'] == 'bullish crossover'].copy()
signal

,Open,High,Low,Close,fast_sma,slow_sma,prev_fast_sma,time,crossover
Date,,,,,,,,,
2018-11-13 00:00:00-05:00,102.880651,104.018982,102.010156,102.297134,103.426871,103.250098,103.119809,2018-11-13 00:00:00-05:00,bullish crossover
2018-11-15 00:00:00-05:00,100.865692,103.565312,99.828123,103.065735,103.468669,103.407494,103.294238,2018-11-15 00:00:00-05:00,bullish crossover
2018-11-16 00:00:00-05:00,102.873575,104.602861,102.604575,104.036041,103.717172,103.518379,103.468669,2018-11-16 00:00:00-05:00,bullish crossover
2018-12-10 00:00:00-05:00,100.683139,103.738219,99.808883,103.363533,104.527931,104.319228,104.093689,2018-12-10 00:00:00-05:00,bullish crossover
2019-02-19 00:00:00-05:00,103.555684,104.391510,103.546074,103.920753,102.453745,102.321621,102.220290,2019-02-19 00:00:00-05:00,bullish crossover
2020-04-14 00:00:00-04:00,165.211872,169.855401,164.234287,169.806519,158.090179,157.113608,156.773373,2020-04-14 00:00:00-04:00,bullish crossover
2020-11-05 00:00:00-05:00,218.193696,220.237667,217.319114,219.422043,206.152953,205.742612,205.327502,2020-11-05 00:00:00-05:00,bullish crossover
2022-08-05 00:00:00-04:00,278.558715,283.049183,278.089709,282.310760,273.267958,273.209835,271.017732,2022-08-05 00:00:00-04:00,bullish crossover


In [58]:
# visualize close price
fig = px.line(df, x=df.index, y=['Close', 'fast_sma', 'slow_sma'],  width=1000, height=400)

for i, row in signal.iterrows():

    fig.add_vline(x=row.time)
    
fig.show()

In [59]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'Open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        
        
class Strategy:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df
        
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
        
    def add_position(self, position):
        self.positions.append(position)
        
        return True
        
# logic
    def run(self):
        for i, data in self.data.iterrows():
            
            if data.crossover == 'bearish crossover':
                for position in self.positions:
                    if position.status == 'Open':
                        position.close_position(data.time, data.Close)
            
            if data.crossover == 'bullish crossover':
                self.add_position(Position(data.time, data.Close, 'buy', self.volume, 0, 0))
        
        return self.get_positions_df()

In [60]:
sma_crossover_strategy = Strategy(df, 10000, 1)
result = sma_crossover_strategy.run()

result

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2018-11-13 00:00:00-05:00,102.297134,buy,1,0,0,2018-11-14 00:00:00-05:00,100.846466,-1.450668,closed,9998.549332
1,2018-11-15 00:00:00-05:00,103.065735,buy,1,0,0,2018-11-19 00:00:00-05:00,100.510216,-2.555519,closed,9995.993813
2,2018-11-16 00:00:00-05:00,104.036041,buy,1,0,0,2018-11-19 00:00:00-05:00,100.510216,-3.525826,closed,9992.467987
3,2018-12-10 00:00:00-05:00,103.363533,buy,1,0,0,2018-12-14 00:00:00-05:00,101.864830,-1.498703,closed,9990.969284
4,2019-02-19 00:00:00-05:00,103.920753,buy,1,0,0,2020-03-16 00:00:00-04:00,132.384552,28.463799,closed,10019.433083
5,2020-04-14 00:00:00-04:00,169.806519,buy,1,0,0,2020-11-03 00:00:00-05:00,202.854095,33.047577,closed,10052.480659
6,2020-11-05 00:00:00-05:00,219.422043,buy,1,0,0,2022-01-18 00:00:00-05:00,300.685455,81.263412,closed,10133.744072
7,2022-08-05 00:00:00-04:00,282.310760,buy,1,0,0,2022-09-02 00:00:00-04:00,256.059998,-26.250763,closed,10107.493309


In [61]:
px.line(result, x='close_datetime', y='pnl')

In [62]:
# visualize close price
fig = px.line(df, x='time', y=['Close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
for i, row in result[result['status'] == 'closed'].iterrows():
    
    if row.profit > 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Green",width=3)
                     )
                      
    elif row.profit < 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Red",width=3)
                      )

    
fig.show()